In [849]:
import csv
import re
import typing as t
from collections import defaultdict
from pathlib import Path

import nltk
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from nltk.corpus import stopwords, wordnet
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from tqdm import tqdm

In [850]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [851]:
DATA_DIR = Path("data/")

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE.upper()} device")

Using CUDA device


In [1059]:
def on_cuda(device: str) -> bool:
    return device == "cuda"


def common_train(
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        train_dataloader: DataLoader,
        epochs: int,
        test_dataloader: DataLoader = None,
        verbose: int = 100,
        on_epoch_end: t.Callable[[], None] = None,
        device: str = "cpu",
) -> t.List[float]:
    train_losses = []
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}\n" + "-" * 32)
        train_loss = train_loop(
            train_dataloader,
            model,
            loss_fn,
            optimizer,
            verbose=verbose,
            device=device,
        )
        train_losses.append(train_loss.item())

        if test_dataloader:
            test_loop(test_dataloader, model, loss_fn, device=device)

        if on_epoch_end:
            on_epoch_end()

        print()
        torch.cuda.empty_cache()
    return train_losses


def train_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        verbose: int = 100,
        device: str = "cpu",
) -> torch.Tensor:
    model.train()

    size = len(dataloader.dataset)  # noqa
    num_batches = len(dataloader)
    avg_loss = 0

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss
        if batch % verbose == 0:
            print(f"loss: {loss:>7f}  [{batch * len(x):>5d}/{size:>5d}]")

        del x, y, pred, loss
        torch.cuda.empty_cache()

    return avg_loss / num_batches


@torch.no_grad()
def test_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        device: str = "cpu",
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    model.eval()

    avg_loss, num_batches = 0, len(dataloader)
    correct, total = 0, 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        avg_loss += loss_fn(pred, y)

        y_test = torch.flatten(y)
        y_pred = torch.flatten(pred.argmax(1))
        total += y_test.size(0)
        correct += (y_pred == y_test).sum()  # noqa

        del x, y, pred
        torch.cuda.empty_cache()

    avg_loss /= num_batches
    accuracy = correct / total
    print(f"Test Error: \n"
          f"\tAccuracy: {accuracy:>4f}, Loss: {avg_loss:>8f}")

    return avg_loss, accuracy


def train_test_split(dataset: t.Union[Dataset, t.Sized], train_part: float) -> t.Tuple[Subset, Subset]:
    train_size = round(train_part * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, lengths=(train_size, test_size))
    return train_dataset, test_dataset


@torch.no_grad()
def get_y_test_y_pred(
        model: nn.Module,
        test_dataloader: DataLoader,
        device: str = "cpu",
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    model.eval()

    y_test = []
    y_pred = []
    for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x).argmax(1)
        y_test.append(y)
        y_pred.append(pred)

        del x
        torch.cuda.empty_cache()

    return torch.flatten(torch.vstack(y_test).detach().cpu()), torch.flatten(torch.vstack(y_pred).detach().cpu())

## 1. Генерирование русских имен при помощи RNN

Датасет: https://disk.yandex.ru/i/2yt18jHUgVEoIw

1.1 На основе файла name_rus.txt создайте датасет.
  * Учтите, что имена могут иметь различную длину
  * Добавьте 4 специальных токена:
    * `<PAD>` для дополнения последовательности до нужной длины;
    * `<UNK>` для корректной обработки ранее не встречавшихся токенов;
    * `<SOS>` для обозначения начала последовательности;
    * `<EOS>` для обозначения конца последовательности.
  * Преобразовывайте строку в последовательность индексов с учетом следующих замечаний:
    * в начало последовательности добавьте токен `<SOS>`;
    * в конец последовательности добавьте токен `<EOS>` и, при необходимости, несколько токенов `<PAD>`;
  * `Dataset.__get_item__` возращает две последовательности: последовательность для обучения и правильный ответ.

  Пример:
  ```
  s = 'The cat sat on the mat'
  # преобразуем в индексы
  s_idx = [2, 5, 1, 2, 8, 4, 7, 3, 0, 0]
  # получаем x и y (__getitem__)
  x = [2, 5, 1, 2, 8, 4, 7, 3, 0]
  y = [5, 1, 2, 8, 4, 7, 3, 0, 0]
  ```


In [1180]:
class NamesVocab:
    PAD = "<PAD>"
    PAD_IDX = 0
    UNK = "<UNK>"
    UNK_IDX = 1
    SOS = "<SOS>"
    SOS_IDX = 2
    EOS = "<EOS>"
    EOS_IDX = 3

    def __init__(self, names: t.List[str]):
        uniques = set()
        max_len = 0
        for name in map(str.lower, names):
            uniques.update(name)
            max_len = max(len(name), max_len)

        self.alphabet = [self.PAD, self.UNK, self.SOS, self.EOS, *uniques]
        self.max_len = max_len + 2  # место для <SOS> и <EOS>

        ch2i = {ch: i for i, ch in enumerate(self.alphabet)}
        self.ch2i = defaultdict(lambda: self.UNK_IDX, ch2i)

    def __len__(self):
        return len(self.alphabet)

    def encode(self, name: str, shift: bool = False) -> torch.Tensor:
        name = [*name, self.EOS]
        if not shift:
            name = [self.SOS, *name]
        indices = [self.ch2i[ch] for ch in name]
        indices += [self.PAD_IDX] * (self.max_len - len(indices))
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices: torch.Tensor) -> str:
        pad_indices = torch.nonzero(indices == self.ch2i[self.PAD], as_tuple=True)[0]
        if len(pad_indices):
            indices = indices[:pad_indices[0]]
        return "".join(self.alphabet[i] for i in indices)


class NamesDataset:
    names: t.List[str]
    vocab: NamesVocab
    data: torch.Tensor
    targets: torch.Tensor

    def __init__(self, path: Path):
        self.names = self.read_names(path)
        self.vocab = NamesVocab(self.names)

        self.data = torch.vstack([self.encode(name, shift=False) for name in self.names])
        self.targets = torch.vstack([self.encode(name, shift=True) for name in self.names])

    def __len__(self):
        return self.data.size(0)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

    @staticmethod
    def read_names(path: Path) -> t.List[str]:
        with open(path) as f:
            return list(map(lambda s: s.strip().lower(), f))

    def encode(self, name: str, shift: bool = False) -> torch.Tensor:
        return self.vocab.encode(name, shift=shift)

    def decode(self, vector: torch.Tensor) -> str:
        return self.vocab.decode(vector)

In [1181]:
names_dataset = NamesDataset(DATA_DIR / "name_rus.txt")
print(f"n: {len(names_dataset)}")
(names_dataset.names[0], *names_dataset[0])

n: 1988


('авдокея',
 tensor([ 2, 31, 21, 26, 30, 23,  8, 20,  3,  0,  0,  0,  0,  0,  0]),
 tensor([31, 21, 26, 30, 23,  8, 20,  3,  0,  0,  0,  0,  0,  0,  0]))

In [1182]:
torch.manual_seed(0)

train_names_dataset, test_names_dataset = train_test_split(names_dataset, train_part=0.8)
print(len(train_names_dataset), len(test_names_dataset))

1590 398


1.2 Создайте и обучите модель для генерации фамилии.

  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`;
  * Используйте рекуррентные слои;
  * Задача ставится как предсказание следующего токена в каждом примере из пакета для каждого момента времени. Т.е. в данный момент времени по текущей подстроке предсказывает следующий символ для данной строки (задача классификации);
  * Примерная схема реализации метода `forward`:
  ```
    input_X: [batch_size x seq_len] -> nn.Embedding -> emb_X: [batch_size x seq_len x embedding_size]
    emb_X: [batch_size x seq_len x embedding_size] -> nn.RNN -> output: [batch_size x seq_len x hidden_size]
    output: [batch_size x seq_len x hidden_size] -> torch.Tensor.reshape -> output: [batch_size * seq_len x hidden_size]
    output: [batch_size * seq_len x hidden_size] -> nn.Linear -> output: [batch_size * seq_len x vocab_size]
  ```

1.3 Напишите функцию, которая генерирует фамилию при помощи обученной модели:
  * Построение начинается с последовательности единичной длины, состоящей из индекса токена `<SOS>`;
  * Начальное скрытое состояние RNN `h_t = None`;
  * В результате прогона последнего токена из построенной последовательности через модель получаете новое скрытое состояние `h_t` и распределение над всеми токенами из словаря;
  * Выбираете 1 токен пропорционально вероятности и добавляете его в последовательность (можно воспользоваться `torch.multinomial`);
  * Повторяете эти действия до тех пор, пока не сгенерирован токен `<EOS>` или не превышена максимальная длина последовательности.

При обучении каждые `k` эпох генерируйте несколько фамилий и выводите их на экран.

In [1183]:
class NamesRNNGenerator(nn.Module):
    _STATE_T = t.Union[t.Optional[torch.Tensor], t.Tuple[t.Optional[torch.Tensor], t.Optional[torch.Tensor]]]
    rnn_state: _STATE_T

    def __init__(
            self,
            num_embeddings: int,
            embedding_dim: int,
            rnn_hidden_size: int,
            rnn_cls: t.Union[t.Type[nn.RNN], t.Type[nn.LSTM], t.Type[nn.GRU]],
    ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        self.rnn = rnn_cls(input_size=embedding_dim, hidden_size=rnn_hidden_size)
        self.fc = nn.Sequential(
            nn.Linear(rnn_hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, num_embeddings),
        )
        self.reset_rnn_state()

    def reset_rnn_state(self):
        self.rnn_state = None

    def keep_rnn_state(self, state: _STATE_T):
        if isinstance(self.rnn, nn.LSTM):
            self.rnn_state = state[0].detach(), state[1].detach()
        else:
            self.rnn_state = state.detach()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)

        x, rnn_state = self.rnn(x, self.rnn_state)
        self.keep_rnn_state(rnn_state)

        x = self.fc(x)
        return x.permute(0, 2, 1)

In [1184]:
def true_prob(pred: torch.Tensor) -> torch.Tensor:
    pred -= pred.min()
    return pred / pred.sum()


def softmax_prob(pred: torch.Tensor) -> torch.Tensor:
    return torch.softmax(pred, 0)


def generate_name(
        model: NamesRNNGenerator,
        dataset: NamesDataset,
        start="",
        prob: t.Callable[[torch.Tensor], torch.Tensor] = None,
        device: str = "cpu",
) -> str:
    len_start = len(start)
    name = dataset.encode(start).to(device)
    name[len_start + 1] = 0  # заменяем <EOS> на <PAD>

    model.eval()
    model.reset_rnn_state()
    for i in range(name.size(0) - len_start - 2):
        pred = model(name.unsqueeze(0)).squeeze()[:, len_start + i]
        if prob:
            next_ch_idx = torch.multinomial(prob(pred), 1)
        else:
            next_ch_idx = pred.argmax()

        if next_ch_idx == NamesVocab.EOS_IDX:
            break
        name[len_start + i + 1] = next_ch_idx

    return dataset.decode(name).replace(NamesVocab.SOS, "")


def on_epoch_end_generate_names(
        model: NamesRNNGenerator,
        dataset: NamesDataset,
) -> t.Callable[[], None]:
    def _on_epoch_end() -> None:
        const = generate_name(model, dataset, device=DEVICE)
        true_random = generate_name(model, dataset, prob=true_prob, device=DEVICE)
        softmax_random = generate_name(model, dataset, prob=softmax_prob, device=DEVICE)
        print(f"\tNames: {const} (max), {true_random} (prob), {softmax_random} (softmax)")

    return _on_epoch_end

In [1185]:
torch.manual_seed(0)

names_gen_net = NamesRNNGenerator(
    num_embeddings=len(names_dataset.vocab),
    embedding_dim=8,
    rnn_hidden_size=64,
    rnn_cls=nn.RNN,
).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(names_gen_net.parameters(), lr=0.001)

train_dataloader = DataLoader(train_names_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_names_dataset, batch_size=128)

In [1186]:
%%time

_ = common_train(
    epochs=20,
    model=names_gen_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    verbose=50,
    on_epoch_end=on_epoch_end_generate_names(names_gen_net, names_dataset),
    device=DEVICE,
)

Epoch 1
--------------------------------
loss: 3.594028  [    0/ 1590]
loss: 2.128230  [  800/ 1590]
Test Error: 
	Accuracy: 0.588945, Loss: 1.591906
	Names: а (max), итуисннарь (prob), лев (softmax)

Epoch 2
--------------------------------
loss: 1.557775  [    0/ 1590]
loss: 1.351737  [  800/ 1590]
Test Error: 
	Accuracy: 0.649246, Loss: 1.252368
	Names: а (max), ергфчшышшю (prob), па (softmax)

Epoch 3
--------------------------------
loss: 1.308698  [    0/ 1590]
loss: 1.181783  [  800/ 1590]
Test Error: 
	Accuracy: 0.669179, Loss: 1.151966
	Names: линя (max), шы (prob), ллира (softmax)

Epoch 4
--------------------------------
loss: 1.068193  [    0/ 1590]
loss: 1.040174  [  800/ 1590]
Test Error: 
	Accuracy: 0.676382, Loss: 1.101062
	Names: а (max), ук (prob), дня (softmax)

Epoch 5
--------------------------------
loss: 1.066873  [    0/ 1590]
loss: 1.104704  [  800/ 1590]
Test Error: 
	Accuracy: 0.678392, Loss: 1.075477
	Names: а (max), ебпукиаырабти (prob), брашвома (softmax)


In [1187]:
y_test, y_pred = get_y_test_y_pred(names_gen_net, test_dataloader, DEVICE)

print(metrics.classification_report(
    y_true=y_test,
    y_pred=y_pred,
    target_names=[names_dataset.vocab.alphabet[i] for i in y_test.unique().sort()[0]],
    zero_division=True,
))

              precision    recall  f1-score   support

       <PAD>       1.00      1.00      1.00      3110
       <EOS>       0.62      0.97      0.76       398
           ф       1.00      0.00      0.00        12
           н       0.25      0.52      0.34       210
           п       1.00      0.00      0.00        36
           ц       1.00      0.00      0.00         1
           е       0.30      0.26      0.28       142
           и       0.18      0.27      0.22       183
           ю       1.00      0.00      0.00        71
           г       1.00      0.00      0.00        33
           й       1.00      0.00      0.00        20
           э       1.00      0.00      0.00         6
           ы       1.00      0.00      0.00        20
           х       1.00      0.00      0.00        46
           б       1.00      0.00      0.00        15
           ш       0.29      0.41      0.34        56
           л       0.44      0.11      0.17       160
           т       0.00    

In [1188]:
print(generate_name(names_gen_net, names_dataset, start="ангел", device=DEVICE))
print(generate_name(names_gen_net, names_dataset, start="ангел", prob=true_prob, device=DEVICE))
print(generate_name(names_gen_net, names_dataset, start="ангел", prob=softmax_prob, device=DEVICE))

ангелиня
ангелдатешца
ангелин


## 2. Генерирование текста при помощи RNN

2.1 Скачайте из интернета какое-нибудь художественное произведение
  * Выбирайте достаточно крупное произведение, чтобы модель лучше обучалась;

2.2 На основе выбранного произведения создайте датасет. 

Отличия от задачи 1:
  * Токены <SOS>, `<EOS>` и `<UNK>` можно не добавлять;
  * При создании датасета текст необходимо предварительно разбить на части. Выберите желаемую длину последовательности `seq_len` и разбейте текст на построки длины `seq_len` (можно без перекрытия, можно с небольшим перекрытием).

2.3 Создайте и обучите модель для генерации текста
  * Задача ставится точно так же как в 1.2;
  * При необходимости можете применить:
    * двухуровневые рекуррентные слои (`num_layers`=2)
    * [обрезку градиентов](https://pytorch.org/docs/stable/generated/torch.nn.utils.clip_grad_norm_.html)

2.4 Напишите функцию, которая генерирует фрагмент текста при помощи обученной модели
  * Процесс генерации начинается с небольшого фрагмента текста `prime`, выбранного вами (1-2 слова) 
  * Сначала вы пропускаете через модель токены из `prime` и генерируете на их основе скрытое состояние рекуррентного слоя `h_t`;
  * После этого вы генерируете строку нужной длины аналогично 1.3
